# Projet APST2 partie 1 : Q-learning/SARSA models on Frozen Lake

## Description of the problem (taken from the website)*
The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world e.g. [3,3] for the 4x4 environment.

Holes in the ice are distributed in set locations when using a pre-determined map or in random locations when a random map is generated.

The player makes moves until they reach the goal or fall in a hole.

#### Action Space
The action shape is (1,) in the range {0, 3} indicating which direction to move the player.

0: Move left

1: Move down

2: Move right

3: Move up
#### Observation Space
The observation is a value representing the player’s current position as current_row * nrows + current_col (where both the row and col start at 0).

For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15. The number of possible observations is dependent on the size of the map.

The observation is returned as an int().

#### Starting State
The episode starts with the player in state [0] (location [0, 0]).

#### Rewards
Reward schedule:

Reach goal: +1

Reach hole: 0

Reach frozen: 0

#### Episode End
The episode ends if the following happens:

Termination:

The player moves into a hole.

The player reaches the goal at max(nrow) * max(ncol) - 1 (location [max(nrow)-1, max(ncol)-1]).

In [2]:
import gymnasium as gym
import pygame

### I.First Look at the frozen Lake game
#### 1.1 Random movements

In [6]:
env = gym.make("FrozenLake-v1", render_mode="human")
state = env.reset()

while True:
    action = env.action_space.sample()  # choose a random action
    next_state, reward, done, truncated ,info = env.step(action)  
    env.render()

    if done:
        print(f"Game over. Reward: {reward}")
        break

Game over. Reward: 0.0


We ran here a code that is performing in each step a random movement (state) that is included in the Discrete(4) space (0,1,2,3) (possible actions) , and of course the random movements will rarely lead to a final reward , as the probability of reaching the reward is very low in this case , this is why we need to use some reoforcement algorithms such as the Q-learning and SARSAA in order to make these algorithms suceed in reaching the rewards.
